# Final model

We recreated a discrete-event simulation (DES) model designed to generate patient arrivals from six different sources and simulate the activity of a 24-bed critical care unit (CCU) admitting and discharging these patients. This model intends to investigate the changes of elective surgery cancellation numbers with the increase of utilised bed counts.

## Aim

The second iteration of the model managed to incorporate the arrival and length of stay distributions for Elective surgery patients, despite the missing data in the Griffith's et al's paper. To replicate the simulation further, we will use LLM prompts to add:
* A warm-up period for the model to enable it to reach a steady state
* A buffer period to avoid the simulation ending before all patients have left the system
* A replication period to run the simulation to obtain an idea of the model's variability. 

Additionally, we aim to assess the impact of how changing the utilised bed counts affects the model's output.


## Prompt Engineering and LLM results:

Griffith's paper (section 2.4) describes that the LOS metric does not correspond to the time a hospital bed in the CCU is occupied. This is because patient length of stay needs to incorporate the changeover time needed to clean the bed and surrounding area for the next patient, and get the resources in a suitable condition for the next patient. This was difficult to recreate as UHW did not provide data on the changeover time to Griffith's et al, but suggested it took between 2-8 hours. An estimate of 5 hours was used, changing the values to the lower and upper end of this distribution. The model will be updated to include this changeover time, as it would be unrealistic for our simulation to assume a bed is immediately available after the previous patient is discharged.

### 5.1 Incorporate a changeover period

### Main command
Update the es_arrivals_generator function for elective surgery patients in python 3.10 and simpy 4.0, to incorporate a changeover time within a specified distribution. All time units are in hours.

### Steps
1. Create a function to get the changeover time, using a truncated normal distribution. 
2. In the `service` method of the class `ElectivePatient`, Add a changeover time function to the LOS timeout before the service is concluded. Do not modify any other code in the service method other than incorporating the LOS timeout, adding the changeover time, freeing the bed and adding the occupancy level.
Another timeout should be added to simulate changeover time, incorporating the specified values. Return a list of bed-occupancy levels.
3. Use a random variable to model the variability in the changeover time. Please return the resulting bed occupancy due to this wait time in the output. 
4. Run the simulation. Print out patient arrival times (indicating day of week, number of weeks). Please return your code integrated into the simulation model.
5. Parameters:
- The changeover times distribution is a normal distribution with a mean of 5, and ranging between 2 and 8. 
- RUN_LENGTH = 12 * 30 * 24 (see the impact of the changeover times over the span of a year/8,640 hours)




The LLM results are documented as follows:

There was an error being returned relating to the distribution(`Exponential`, `Lognormal`) and `Patient` and `ElectivePatient` Classes not being defined in the code, so this was manually added alongside the prompt. Code was manually added so the simulation could run.

In [41]:
import simpy
import numpy as np
import itertools
from scipy.stats import lognorm, truncnorm
import math
import random

# Helper function to print out messages    
def trace(msg):
    '''
    Turning printing of events on and off.
    
    Params:
    -------
    msg: str
        string to print to screen.
    '''
    if TRACE:
        print(msg)

class Scenario:
    '''
    Parameter class for CCU simulation model inputs.
    '''
    def __init__(self):
        '''
        The init method sets up our defaults. 
        '''
        self.beds = simpy.Resource(env, capacity=N_BEDS)
        
        # Inter-arrival time (IAT) distributions for five types of patients
        self.ae_arrival_dist = Exponential(MEAN_IAT_ae, random_seed=SEEDS[0])
        self.ward_arrival_dist = Exponential(MEAN_IAT_ward, random_seed=SEEDS[1])
        self.emer_arrival_dist = Exponential(MEAN_IAT_emer, random_seed=SEEDS[2])
        self.oth_arrival_dist = Exponential(MEAN_IAT_oth, random_seed=SEEDS[3])
        self.xray_arrival_dist = Exponential(MEAN_IAT_xray, random_seed=SEEDS[4])

        # Length of stay (LOS) distributions for six types of patients
        self.ae_los_dist = Lognormal(MEAN_LOS_ae, STD_LOS_ae, random_seed=SEEDS[5])
        self.ward_los_dist = Lognormal(MEAN_LOS_ward, STD_LOS_ward, random_seed=SEEDS[6])
        self.emer_los_dist = Lognormal(MEAN_LOS_emer, STD_LOS_emer, random_seed=SEEDS[7])
        self.oth_los_dist = Lognormal(MEAN_LOS_oth, STD_LOS_oth, random_seed=SEEDS[8])
        self.xray_los_dist = Lognormal(MEAN_LOS_xray, STD_LOS_xray, random_seed=SEEDS[9])
        self.es_los_dist = Lognormal(MEAN_LOS_es, STD_LOS_es, random_seed=SEEDS[10])
        
# Custom classes for distributions
class Exponential:
    def __init__(self, mean, random_seed=None):
        self.mean = mean
        self.rng = np.random.default_rng(seed=random_seed)

    def sample(self):
        return self.rng.exponential(self.mean)

class Lognormal:
    def __init__(self, mean, std, random_seed=None):
        self.mean = mean
        self.std = std
        self.rng = np.random.default_rng(seed=random_seed)
        self.sigma = np.sqrt(np.log(1 + (std/mean)**2))
        self.scale = np.exp(np.log(mean) - 0.5*self.sigma**2)

    def sample(self):
        return self.rng.lognormal(np.log(self.scale), self.sigma)
    

# Function to generate changeover time with truncated normal distribution
def get_changeover_time(mean, min_val, max_val):
    """Generate changeover time using a truncated normal distribution."""
    while True:
        changeover_time = random.normalvariate(mean, (max_val - min_val) / 2)
        if min_val <= changeover_time <= max_val:
            return changeover_time

# Sample function for elective surgery
def sample_daily_arrival_times(mean, std, lower_bound, upper_bound, sample_size, random_seed):
    """
    Sample daily arrival times from a truncated normal distribution.
    """
    np.random.seed(random_seed)
    a, b = (lower_bound - mean) / std, (upper_bound - mean) / std
    daily_arrival_times = truncnorm.rvs(a, b, loc=mean, scale=std, size=sample_size)
    daily_arrival_times.sort()
    return daily_arrival_times


    
# Patient class
class Patient:
    def __init__(self, env, patient_id, source, los_dist):
        self.env = env
        self.patient_id = patient_id
        self.source = source
        self.los_dist = los_dist

    def service(self, ccu):
        arrive_time = self.env.now
        print(f"Patient {self.patient_id} from {self.source} arrived at {arrive_time:.2f}")

        with ccu.request() as request:
            yield request
            wait_time = self.env.now - arrive_time
            los = self.los_dist.sample()
            print(f"Patient {self.patient_id} from {self.source} waited for {wait_time:.2f} hours, LOS: {los:.2f}")
            yield self.env.timeout(los)
            print(f"Patient {self.patient_id} from {self.source} left at {self.env.now:.2f}")

# ElectivePatient class                    
class ElectivePatient(Patient):
    def __init__(self, env, patient_id, source, los_dist):
        super().__init__(env, patient_id, source, los_dist)
        self.cancelled_surgeries = []  # Track patients who cancel surgery

    # Updated service method to include changeover period
    def service(self, ccu):
        arrive_time = self.env.now
        print(f"Patient {self.patient_id} from {self.source} arrived at {arrive_time:.2f}")

        if ccu.count < ccu.capacity:
            with ccu.request() as request:
                yield request
                los = self.los_dist.sample()
                print(f"Patient {self.patient_id} from {self.source} admitted with LOS: {los:.2f}")
                yield self.env.timeout(los)
                
                # Simulate changeover time
                changeover_time = get_changeover_time(5, 2, 8)
                yield self.env.timeout(changeover_time)
                
                # Record the bed occupancy level
                bed_occupancy_level = ccu.count - 1  # Subtracting 1 because this bed will now be free
                print(f"Bed occupancy after patient {self.patient_id} departure: {bed_occupancy_level}")
                
                # Free the bed
                ccu.release(request)
                
                # This method does not return a value, but you can add to a list or handle occupancy level as needed
        else:
            self.cancelled_surgeries.append(self.patient_id)
            print(f"Patient {self.patient_id} from {self.source} cancelled at {arrive_time:.2f}")

# CCU class
class CCU:
    def __init__(self, env, capacity):
        self.env = env
        self.resource = simpy.Resource(env, capacity=capacity)

    def ae_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "ae", los_dist)
            self.env.process(patient.service(self.resource))

    def ward_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "ward", los_dist)
            self.env.process(patient.service(self.resource))

    def emer_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "emer", los_dist)
            self.env.process(patient.service(self.resource))

    def oth_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "oth", los_dist)
            self.env.process(patient.service(self.resource))

    def xray_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "xray", los_dist)
            self.env.process(patient.service(self.resource))
            
    def es_arrivals_generator(self, num_weeks, mean, std, lower_bound, upper_bound, daily_sample_size, los_dist):
        while True:
            for week in range(num_weeks):
                for day_of_week in range(7):  # Loop through each day of the week
                    current_day = week * 7 + day_of_week  # Calculate the absolute day 

                    if 0 <= day_of_week <= 4:  # Check for weekdays
                        # Adjust random seed based on week and day for variety
                        random_seed = week * 7 + day_of_week
                        daily_arrival_times = sample_daily_arrival_times(mean, std, lower_bound, upper_bound, 
                                                                         daily_sample_size, random_seed)

                        last_arrival_time = current_day * 24  # Convert current day to hours
                        for arrival_time in daily_arrival_times:
                            actual_arrival_time = last_arrival_time + arrival_time
                            yield self.env.timeout(actual_arrival_time - env.now)
                            patient_id = next(patient_id_generator)
                            patient = ElectivePatient(self.env, patient_id, "es", los_dist)
                            self.env.process(patient.service(self.resource))
                    else:
                        continue

    
    
# Simulation parameters
RUN_LENGTH = 12 * 30 * 24  # 12 months
N_BEDS = 24
patient_id_generator = itertools.count()

# Elective surgery parameters
daily_sample_size = int(1182 / 12 / 30 / 3)  
mean = 17.91
std = 3.16
lower_bound = 0
upper_bound = 24
num_weeks = math.ceil(RUN_LENGTH / 24 / 7)


# Initialize simulation environment
env = simpy.Environment()
ccu = CCU(env, N_BEDS)

# Patient arrival source distributions
IAT_DISTRIBUTIONS = {
    "ae": Exponential(22.72),
    "ward": Exponential(26.0),
    "emer": Exponential(37.0),
    "oth": Exponential(47.2),
    "xray": Exponential(575.0)
}


# Patient LOS distributions
LOS_DISTRIBUTIONS = {
    "ae": Lognormal(128.79, 267.51),
    "ward": Lognormal(177.89, 276.54),
    "emer": Lognormal(140.15, 218.02),
    "oth": Lognormal(212.86, 457.67),
    "xray": Lognormal(87.53, 108.15),
    "es": Lognormal(57.34, 99.78)
}


# Starting the process
env.process(ccu.ae_arrivals_generator(IAT_DISTRIBUTIONS["ae"], LOS_DISTRIBUTIONS["ae"]))
env.process(ccu.ward_arrivals_generator(IAT_DISTRIBUTIONS["ward"], LOS_DISTRIBUTIONS["ward"]))
env.process(ccu.emer_arrivals_generator(IAT_DISTRIBUTIONS["emer"], LOS_DISTRIBUTIONS["emer"]))
env.process(ccu.oth_arrivals_generator(IAT_DISTRIBUTIONS["oth"], LOS_DISTRIBUTIONS["oth"]))
env.process(ccu.xray_arrivals_generator(IAT_DISTRIBUTIONS["xray"], LOS_DISTRIBUTIONS["xray"]))
env.process(ccu.es_arrivals_generator(num_weeks, mean, std, lower_bound, upper_bound, 
                                      daily_sample_size, LOS_DISTRIBUTIONS["es"]))

# Run the simulation
env.run(until=RUN_LENGTH)



Patient 0 from ae arrived at 0.08
Patient 0 from ae waited for 0.00 hours, LOS: 31.06
Patient 1 from ward arrived at 8.40
Patient 1 from ward waited for 0.00 hours, LOS: 256.19
Patient 2 from emer arrived at 13.39
Patient 2 from emer waited for 0.00 hours, LOS: 7.44
Patient 3 from ae arrived at 14.97
Patient 3 from ae waited for 0.00 hours, LOS: 12.09
Patient 4 from emer arrived at 15.85
Patient 4 from emer waited for 0.00 hours, LOS: 23.77
Patient 5 from es arrived at 18.18
Patient 5 from es admitted with LOS: 31.10
Patient 6 from ward arrived at 20.66
Patient 6 from ward waited for 0.00 hours, LOS: 70.62
Patient 2 from emer left at 20.84
Patient 7 from oth arrived at 22.16
Patient 7 from oth waited for 0.00 hours, LOS: 49.23
Patient 3 from ae left at 27.07
Patient 0 from ae left at 31.14
Patient 8 from ae arrived at 32.51
Patient 8 from ae waited for 0.00 hours, LOS: 75.57
Patient 9 from emer arrived at 33.47
Patient 9 from emer waited for 0.00 hours, LOS: 103.18
Patient 10 from emer

The function provided by the LLM to account for changeover time can run simulation successfully when the existing and new code is consolidated together.

Now, the updated code will utilise the performance metrics functionality from the previous iteration, specifically the `Auditor` Class. This will allow us to calculate the impact that the changeover time has on the bed occupancy levels and LOS.

In [42]:
import pandas as pd

class Auditor:
    def __init__(self, env, run_length, bed_counts, first_obs=None, interval=None):
        '''
        Auditor Constructor
        
        Params:
        -----
        env: simpy.Environment
            
        first_obs: float, optional (default=None)
            Time of first scheduled observation.  If none then no scheduled
            audit will take place
        
        interval: float, optional (default=None)
            Time period between scheduled observations. If none then no scheduled
            audit will take place
        '''
        self.env = env
        self.first_observation = first_obs
        self.interval = interval
        self.run_length = run_length
        self.bed_counts = bed_counts
        
        self.queues = []
        self.services = []
        
        # dict to hold states
        self.metrics = {}
        
        # scheduled the periodic audits
        if not first_obs is None:
            env.process(self.scheduled_observation())
            env.process(self.process_end_of_run())
            
    def add_resource_to_audit(self, resource, name, audit_type='qs'):
        if 'q' in audit_type:
            self.queues.append((name, resource))
            self.metrics[f'queue_length_{name}'] = []
        
        if 's' in audit_type:
            self.services.append((name, resource))
            self.metrics[f'occupied_{name}'] = []   
                    
    def record_queue_length(self):
        for name, res in self.queues:
            self.metrics[f'queue_length_{name}'].append(len(res.queue)) 
               
    def record_occupied_bed(self):
        for name, res in self.services:
            self.metrics[f'occupied_{name}'].append(res.count) 

            
    def scheduled_observation(self):
        '''
        simpy process to control the frequency of 
        auditor observations of the model.  
        
        The first observation takes place at self.first_obs
        and subsequent observations are spaced self.interval
        apart in time.
        '''
        # delay first observation
        yield self.env.timeout(self.first_observation)
        self.record_queue_length()
        self.record_occupied_bed()
        
        while True:
            yield self.env.timeout(self.interval)
            self.record_queue_length()
            self.record_occupied_bed()
               
        
    def process_end_of_run(self):
        '''
        Create an end of run summary
        
        Returns:
        ---------
            pd.DataFrame
        '''
        
        yield self.env.timeout(self.run_length - 1)
        
        run_results = {}

        for name, res in self.queues:
            queue_length = np.array(self.metrics[f'queue_length_{name}'])
            run_results[f'mean_queue_{name}'] = queue_length.mean()
            
        for name, res in self.services:
            serviced_beds = np.array(self.metrics[f'occupied_{name}'])
            run_results[f'mean_occupied_{name}'] = serviced_beds.mean()
            run_results[f'occupancy_rate_{name}'] = (serviced_beds.mean() / self.bed_counts) 

        self.summary_frame = pd.Series(run_results).to_frame()
        self.summary_frame.columns = ['estimate']      

In [43]:
# collect performance metrics
def run_results(model, auditor):
    df_results = auditor.summary_frame
    
    # admissions from various sources
    ae_admissions = sum(patient.source == 'A&E' for patient in model.patients)
    ward_admissions = sum(patient.source == 'Ward' for patient in model.patients)
    emer_admissions = sum(patient.source == 'Emergency' for patient in model.patients)
    oth_admissions = sum(patient.source == 'Other Hospital' for patient in model.patients)
    xray_admissions = sum(patient.source == 'X-ray' for patient in model.patients)
    
    # Calculate the number of cancelled elective surgery patients
    cancelled_es = len(ElectivePatient.cancelled_surgeries)
    es_admissions = sum(patient.source == 'Elective Surgery' for patient in model.patients) - cancelled_es
    
    # total admissions
    total_admissions = len(model.patients) - cancelled_es
        
    # waiting time = sum(waiting times) / no. patients
    mean_wait_time = np.array([patient.wait_time 
                                for patient in model.patients]).mean()
    
    # bed days utilisation = sum(los) / (run length X no. beds)
    bed_day_util = np.array([patient.los 
                     for patient in model.patients]).sum() / \
                    (RUN_LENGTH * N_BEDS)

    # append to results df
    new_row = pd.DataFrame({'estimate':{'Total_admissions': total_admissions,
                                        'A&E_admissions': ae_admissions,
                                        'Ward_admissions': ward_admissions,
                                        'Emergency_admissions': emer_admissions,
                                        'Other_hospital_admissions': oth_admissions,
                                        'Xray_admissions': xray_admissions,
                                        'Elective_Surgery_admissions': es_admissions,
                                        'Cancelled_Surgeries': cancelled_es, 
                                        'mean_wait_hours': mean_wait_time, 
                                        'bed_days_util': bed_day_util}})

    df_results = pd.concat([df_results, new_row])
    return df_results
    

# Bug Fixes

1.1 The code initially ran with an error as CCU class was instantiated with its default_args, rather defining the correct number of beds when creating the model. As N_BEDS (24) is the specified value for capacity, passing this variable into the class resolved the issue.

*In the correct instantiation, N_BEDS should be passed instead of default_args:*

model = CCU(env, N_BEDS)  

1.2 The required arguments must be provided for `iat_dist` and `los_dist` in each patient class.


In [47]:
# Run the simulation model
##########################################################
# scheduled audit intervals in hours.
FIRST_OBS = 24
OBS_INTERVAL = 48
N_BEDS = 24

# RUN lENGTH
RUN_LENGTH = 12 * 30 * 24  # 12 months

# Elective surgery parameters
DAILY_SAMPLE_SIZE = int(1182 / 12 / 30 / 3 )  
NUM_WEEKS = math.ceil(RUN_LENGTH / 24 / 7)

# Turn off tracing
TRACE = False
##########################################################

# generate patient identifier
identifier_generator = itertools.count()

# create simpy environment
env = simpy.Environment()

# base case scenario with default parameters
default_args = Scenario()

# instantiate an auditor
auditor = Auditor(env, RUN_LENGTH, N_BEDS, FIRST_OBS, OBS_INTERVAL)
auditor.add_resource_to_audit(default_args.beds, 'beds')

# create the model
model = CCU(env, N_BEDS)


# Simulation parameters
RUN_LENGTH = 12 * 30 * 24  # 12 months
N_BEDS = 24
patient_id_generator = itertools.count()

# Elective surgery parameters
daily_sample_size = int(1182 / 12 / 30 / 3)  
mean = 17.91
std = 3.16
lower_bound = 0
upper_bound = 24
num_weeks = math.ceil(RUN_LENGTH / 24 / 7)


# Initialize simulation environment
env = simpy.Environment()
ccu = CCU(env, N_BEDS)

# Patient arrival source distributions
IAT_DISTRIBUTIONS = {
    "ae": Exponential(22.72),
    "ward": Exponential(26.0),
    "emer": Exponential(37.0),
    "oth": Exponential(47.2),
    "xray": Exponential(575.0)
}


# Patient LOS distributions
LOS_DISTRIBUTIONS = {
    "ae": Lognormal(128.79, 267.51),
    "ward": Lognormal(177.89, 276.54),
    "emer": Lognormal(140.15, 218.02),
    "oth": Lognormal(212.86, 457.67),
    "xray": Lognormal(87.53, 108.15),
    "es": Lognormal(57.34, 99.78)
}


# Starting the process
env.process(model.ae_arrivals_generator(IAT_DISTRIBUTIONS["ae"], LOS_DISTRIBUTIONS["ae"]))
env.process(model.ward_arrivals_generator(IAT_DISTRIBUTIONS["ward"], LOS_DISTRIBUTIONS["ward"]))
env.process(model.emer_arrivals_generator(IAT_DISTRIBUTIONS["emer"], LOS_DISTRIBUTIONS["emer"]))
env.process(model.oth_arrivals_generator(IAT_DISTRIBUTIONS["oth"], LOS_DISTRIBUTIONS["oth"]))
env.process(model.xray_arrivals_generator(IAT_DISTRIBUTIONS["xray"], LOS_DISTRIBUTIONS["xray"]))
env.process(model.es_arrivals_generator(num_weeks, mean, std, lower_bound, upper_bound, 
                                      daily_sample_size, LOS_DISTRIBUTIONS["es"]))


# Remember to add the end of run process for the auditor
end_of_run_process = env.process(auditor.process_end_of_run())

# run the simulation
env.run(until=RUN_LENGTH)

# after the run, access the summary frame
print(f'End of run. simulation clock time = {env.now}')

#checks if summary_frame has been set


print('\nSingle run results\n------------------')
df_results = auditor.summary_frame
print(df_results.round(2))

       

End of run. simulation clock time = 8640
The summary_frame has not been set. Check the process_end_of_run method.


### Original code of the simulation model: Imports
 ```python
import simpy
import numpy as np
import itertools
from scipy.stats import lognorm, truncnorm
import math


# Custom classes for distributions
class Exponential:
    def __init__(self, mean, random_seed=None):
        self.mean = mean
        self.rng = np.random.default_rng(seed=random_seed)

    def sample(self):
        return self.rng.exponential(self.mean)

class Lognormal:
    def __init__(self, mean, std, random_seed=None):
        self.mean = mean
        self.std = std
        self.rng = np.random.default_rng(seed=random_seed)
        self.sigma = np.sqrt(np.log(1 + (std/mean)**2))
        self.scale = np.exp(np.log(mean) - 0.5*self.sigma**2)

    def sample(self):
        return self.rng.lognormal(np.log(self.scale), self.sigma)
    

# Sample function for elective surgery
def sample_daily_arrival_times(mean, std, lower_bound, upper_bound, sample_size, random_seed):
    """
    Sample daily arrival times from a truncated normal distribution.
    """
    np.random.seed(random_seed)
    a, b = (lower_bound - mean) / std, (upper_bound - mean) / std
    daily_arrival_times = truncnorm.rvs(a, b, loc=mean, scale=std, size=sample_size)
    daily_arrival_times.sort()
    return daily_arrival_times

    
# Patient class
class Patient:
    def __init__(self, env, patient_id, source, los_dist):
        self.env = env
        self.patient_id = patient_id
        self.source = source
        self.los_dist = los_dist

    def service(self, ccu):
        arrive_time = self.env.now
        print(f"Patient {self.patient_id} from {self.source} arrived at {arrive_time:.2f}")

        with ccu.request() as request:
            yield request
            wait_time = self.env.now - arrive_time
            los = self.los_dist.sample()
            print(f"Patient {self.patient_id} from {self.source} waited for {wait_time:.2f} hours, LOS: {los:.2f}")
            yield self.env.timeout(los)
            print(f"Patient {self.patient_id} from {self.source} left at {self.env.now:.2f}")
            
            
# ElectivePatient class            
class ElectivePatient(Patient):
    def __init__(self, env, patient_id, source, los_dist):
        super().__init__(env, patient_id, source, los_dist)
        self.cancelled_surgeries = []  # Track patients who cancel surgery
    
    def service(self, ccu):
        arrive_time = self.env.now
        print(f"Patient {self.patient_id} from {self.source} arrived at {arrive_time:.2f}")

        if ccu.count < ccu.capacity:
            with ccu.request() as request:
                yield request
                los = self.los_dist.sample()
                print(f"Patient {self.patient_id} from {self.source} admitted with LOS: {los:.2f}")
                yield self.env.timeout(los)
                print(f"Patient {self.patient_id} from {self.source} left at {self.env.now:.2f}")
        else:
            self.cancelled_surgeries.append(self.patient_id)
            print(f"Patient {self.patient_id} from {self.source} cancelled at {arrive_time:.2f}")
            

# CCU class
class CCU:
    def __init__(self, env, capacity):
        self.env = env
        self.resource = simpy.Resource(env, capacity=capacity)

    def ae_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "ae", los_dist)
            self.env.process(patient.service(self.resource))

    def ward_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "ward", los_dist)
            self.env.process(patient.service(self.resource))

    def emer_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "emer", los_dist)
            self.env.process(patient.service(self.resource))

    def oth_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "oth", los_dist)
            self.env.process(patient.service(self.resource))

    def xray_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "xray", los_dist)
            self.env.process(patient.service(self.resource))
            
    def es_arrivals_generator(self, num_weeks, mean, std, lower_bound, upper_bound, daily_sample_size, los_dist):
        while True:
            for week in range(num_weeks):
                for day_of_week in range(7):  # Loop through each day of the week
                    current_day = week * 7 + day_of_week  # Calculate the absolute day 

                    if 0 <= day_of_week <= 4:  # Check for weekdays
                        # Adjust random seed based on week and day for variety
                        random_seed = week * 7 + day_of_week
                        daily_arrival_times = sample_daily_arrival_times(mean, std, lower_bound, upper_bound, 
                                                                         daily_sample_size, random_seed)

                        last_arrival_time = current_day * 24  # Convert current day to hours
                        for arrival_time in daily_arrival_times:
                            actual_arrival_time = last_arrival_time + arrival_time
                            yield self.env.timeout(actual_arrival_time - env.now)
                            patient_id = next(patient_id_generator)
                            patient = ElectivePatient(self.env, patient_id, "es", los_dist)
                            self.env.process(patient.service(self.resource))
                    else:
                        continue

    
    
# Simulation parameters
RUN_LENGTH = 12 * 30 * 24  # 12 months
N_BEDS = 24
patient_id_generator = itertools.count()

# Elective surgery parameters
daily_sample_size = int(1182 / 12 / 30 / 3)  
mean = 17.91
std = 3.16
lower_bound = 0
upper_bound = 24
num_weeks = math.ceil(RUN_LENGTH / 24 / 7)


# Initialize simulation environment
env = simpy.Environment()
ccu = CCU(env, N_BEDS)

# Patient arrival source distributions
IAT_DISTRIBUTIONS = {
    "ae": Exponential(22.72),
    "ward": Exponential(26.0),
    "emer": Exponential(37.0),
    "oth": Exponential(47.2),
    "xray": Exponential(575.0)
}


# Patient LOS distributions
LOS_DISTRIBUTIONS = {
    "ae": Lognormal(128.79, 267.51),
    "ward": Lognormal(177.89, 276.54),
    "emer": Lognormal(140.15, 218.02),
    "oth": Lognormal(212.86, 457.67),
    "xray": Lognormal(87.53, 108.15),
    "es": Lognormal(57.34, 99.78)
}


# Starting the process
env.process(ccu.ae_arrivals_generator(IAT_DISTRIBUTIONS["ae"], LOS_DISTRIBUTIONS["ae"]))
env.process(ccu.ward_arrivals_generator(IAT_DISTRIBUTIONS["ward"], LOS_DISTRIBUTIONS["ward"]))
env.process(ccu.emer_arrivals_generator(IAT_DISTRIBUTIONS["emer"], LOS_DISTRIBUTIONS["emer"]))
env.process(ccu.oth_arrivals_generator(IAT_DISTRIBUTIONS["oth"], LOS_DISTRIBUTIONS["oth"]))
env.process(ccu.xray_arrivals_generator(IAT_DISTRIBUTIONS["xray"], LOS_DISTRIBUTIONS["xray"]))
env.process(ccu.es_arrivals_generator(num_weeks, mean, std, lower_bound, upper_bound, 
                                      daily_sample_size, LOS_DISTRIBUTIONS["es"]))

# Run the simulation
env.run(until=RUN_LENGTH)



In [ ]:
import simpy
import numpy as np
import itertools
from scipy.stats import lognorm, truncnorm
import math


# Custom classes for distributions
class Exponential:
    def __init__(self, mean, random_seed=None):
        self.mean = mean
        self.rng = np.random.default_rng(seed=random_seed)

    def sample(self):
        return self.rng.exponential(self.mean)

class Lognormal:
    def __init__(self, mean, std, random_seed=None):
        self.mean = mean
        self.std = std
        self.rng = np.random.default_rng(seed=random_seed)
        self.sigma = np.sqrt(np.log(1 + (std/mean)**2))
        self.scale = np.exp(np.log(mean) - 0.5*self.sigma**2)

    def sample(self):
        return self.rng.lognormal(np.log(self.scale), self.sigma)
    

# Sample function for elective surgery
def sample_daily_arrival_times(mean, std, lower_bound, upper_bound, sample_size, random_seed):
    """
    Sample daily arrival times from a truncated normal distribution.
    """
    np.random.seed(random_seed)
    a, b = (lower_bound - mean) / std, (upper_bound - mean) / std
    daily_arrival_times = truncnorm.rvs(a, b, loc=mean, scale=std, size=sample_size)
    daily_arrival_times.sort()
    return daily_arrival_times

    
# Patient class
class Patient:
    def __init__(self, env, patient_id, source, los_dist):
        self.env = env
        self.patient_id = patient_id
        self.source = source
        self.los_dist = los_dist

    def service(self, ccu):
        arrive_time = self.env.now
        print(f"Patient {self.patient_id} from {self.source} arrived at {arrive_time:.2f}")

        with ccu.request() as request:
            yield request
            wait_time = self.env.now - arrive_time
            los = self.los_dist.sample()
            print(f"Patient {self.patient_id} from {self.source} waited for {wait_time:.2f} hours, LOS: {los:.2f}")
            yield self.env.timeout(los)
            print(f"Patient {self.patient_id} from {self.source} left at {self.env.now:.2f}")
            
            
# ElectivePatient class            
class ElectivePatient(Patient):
    def __init__(self, env, patient_id, source, los_dist):
        super().__init__(env, patient_id, source, los_dist)
        self.cancelled_surgeries = []  # Track patients who cancel surgery
    
    def service(self, ccu):
        arrive_time = self.env.now
        print(f"Patient {self.patient_id} from {self.source} arrived at {arrive_time:.2f}")

        if ccu.count < ccu.capacity:
            with ccu.request() as request:
                yield request
                los = self.los_dist.sample()
                print(f"Patient {self.patient_id} from {self.source} admitted with LOS: {los:.2f}")
                yield self.env.timeout(los)
                print(f"Patient {self.patient_id} from {self.source} left at {self.env.now:.2f}")
        else:
            self.cancelled_surgeries.append(self.patient_id)
            print(f"Patient {self.patient_id} from {self.source} cancelled at {arrive_time:.2f}")
            

# CCU class
class CCU:
    def __init__(self, env, capacity):
        self.env = env
        self.resource = simpy.Resource(env, capacity=capacity)

    def ae_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "ae", los_dist)
            self.env.process(patient.service(self.resource))

    def ward_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "ward", los_dist)
            self.env.process(patient.service(self.resource))

    def emer_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "emer", los_dist)
            self.env.process(patient.service(self.resource))

    def oth_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "oth", los_dist)
            self.env.process(patient.service(self.resource))

    def xray_arrivals_generator(self, iat_dist, los_dist):
        while True:
            yield self.env.timeout(iat_dist.sample())
            patient_id = next(patient_id_generator)
            patient = Patient(self.env, patient_id, "xray", los_dist)
            self.env.process(patient.service(self.resource))
            
    def es_arrivals_generator(self, num_weeks, mean, std, lower_bound, upper_bound, daily_sample_size, los_dist):
        while True:
            for week in range(num_weeks):
                for day_of_week in range(7):  # Loop through each day of the week
                    current_day = week * 7 + day_of_week  # Calculate the absolute day 

                    if 0 <= day_of_week <= 4:  # Check for weekdays
                        # Adjust random seed based on week and day for variety
                        random_seed = week * 7 + day_of_week
                        daily_arrival_times = sample_daily_arrival_times(mean, std, lower_bound, upper_bound, 
                                                                         daily_sample_size, random_seed)

                        last_arrival_time = current_day * 24  # Convert current day to hours
                        for arrival_time in daily_arrival_times:
                            actual_arrival_time = last_arrival_time + arrival_time
                            yield self.env.timeout(actual_arrival_time - env.now)
                            patient_id = next(patient_id_generator)
                            patient = ElectivePatient(self.env, patient_id, "es", los_dist)
                            self.env.process(patient.service(self.resource))
                    else:
                        continue

    
    
# Simulation parameters
RUN_LENGTH = 12 * 30 * 24  # 12 months
N_BEDS = 24
patient_id_generator = itertools.count()

# Elective surgery parameters
daily_sample_size = int(1182 / 12 / 30 / 3)  
mean = 17.91
std = 3.16
lower_bound = 0
upper_bound = 24
num_weeks = math.ceil(RUN_LENGTH / 24 / 7)


# Initialize simulation environment
env = simpy.Environment()
ccu = CCU(env, N_BEDS)

# Patient arrival source distributions
IAT_DISTRIBUTIONS = {
    "ae": Exponential(22.72),
    "ward": Exponential(26.0),
    "emer": Exponential(37.0),
    "oth": Exponential(47.2),
    "xray": Exponential(575.0)
}


# Patient LOS distributions
LOS_DISTRIBUTIONS = {
    "ae": Lognormal(128.79, 267.51),
    "ward": Lognormal(177.89, 276.54),
    "emer": Lognormal(140.15, 218.02),
    "oth": Lognormal(212.86, 457.67),
    "xray": Lognormal(87.53, 108.15),
    "es": Lognormal(57.34, 99.78)
}


# Starting the process
env.process(ccu.ae_arrivals_generator(IAT_DISTRIBUTIONS["ae"], LOS_DISTRIBUTIONS["ae"]))
env.process(ccu.ward_arrivals_generator(IAT_DISTRIBUTIONS["ward"], LOS_DISTRIBUTIONS["ward"]))
env.process(ccu.emer_arrivals_generator(IAT_DISTRIBUTIONS["emer"], LOS_DISTRIBUTIONS["emer"]))
env.process(ccu.oth_arrivals_generator(IAT_DISTRIBUTIONS["oth"], LOS_DISTRIBUTIONS["oth"]))
env.process(ccu.xray_arrivals_generator(IAT_DISTRIBUTIONS["xray"], LOS_DISTRIBUTIONS["xray"]))
env.process(ccu.es_arrivals_generator(num_weeks, mean, std, lower_bound, upper_bound, 
                                      daily_sample_size, LOS_DISTRIBUTIONS["es"]))

# Run the simulation
env.run(until=RUN_LENGTH)


Patient 0 from oth arrived at 10.75
Patient 0 from oth waited for 0.00 hours, LOS: 201.59
Patient 1 from emer arrived at 14.55
Patient 1 from emer waited for 0.00 hours, LOS: 199.92
Patient 2 from oth arrived at 17.83
Patient 2 from oth waited for 0.00 hours, LOS: 100.75
Patient 3 from es arrived at 18.18
Patient 3 from es admitted with LOS: 11.68
Patient 4 from ae arrived at 18.51
Patient 4 from ae waited for 0.00 hours, LOS: 339.70
Patient 5 from emer arrived at 19.27
Patient 5 from emer waited for 0.00 hours, LOS: 43.21
Patient 3 from es left at 29.86
Patient 6 from emer arrived at 30.62
Patient 6 from emer waited for 0.00 hours, LOS: 576.35
Patient 7 from ae arrived at 30.89
Patient 7 from ae waited for 0.00 hours, LOS: 78.31
Patient 8 from ward arrived at 36.38
Patient 8 from ward waited for 0.00 hours, LOS: 88.37
Patient 9 from es arrived at 41.16
Patient 9 from es admitted with LOS: 41.22
Patient 10 from ae arrived at 51.35
Patient 10 from ae waited for 0.00 hours, LOS: 177.88
P




## Model Description

In the paper published by Griffiths et al. (2010), the simulation model of CCU admits patients from two different routes: `"unplanned admission"` and `"planned admission"`. In the former route, patients from five various sources have to wait until a bed is available, then they can enter the unit and stay here for a specific period of time to receive therapy. After the treatment is completed, they will leave the system immediately. In the latter route, patients from elective surgery route can only enter the system when there are unoccupied beds, or they have to cancel the surgery if no bed is available at present. After their admission and treatment, they will leave the system immediately. 


## Parameters
The input parameters that configure each simulation are:

**Inter-arrival time, IAT & daily arrivals**

| Source        	| Distribution 	| Mean (hours) 	| Standard Dev (hours) 	|
|-------------------|--------------	|-------------	|-----------------------|
|  A&E              | Exponential  	| 22.72         |                       |
|  Ward             | Exponential  	| 26.0          |                       |
|  Emergency        | Exponential  	| 37.0          |                       |
|  Other hospitals  | Exponential  	| 47.2          |                       |
|  X-Ray            | Exponential  	| 575.0         |                       |
|  Elective, daily arrivals         | Normal    	| 17.91         |  3.16                 |

  
**Length of stay, LOS**

| Source        	| Distribution 	| Mean (hours) 	| Standard Dev (hours) 	|
|-------------------|--------------	|-------------	|-----------------------|
|  A&E              | Lognormal  	| 128.79        |  267.51               |
|  Ward             | Lognormal  	| 177.89        |  276.54               |
|  Emergency        | Lognormal  	| 140.15        |  218.02               |
|  Other hospitals  | Lognormal  	| 212.86        |  457.67               |
|  X-Ray            | Lognormal  	| 87.53         |  108.15               |
|  Elective         | Lognormal    	| 57.34         |  99.78                |


**Changeover time**
-	Changeover time:  After each discharge, the time required for bed and the surrounding area cleaning (hours).                              


## Environment 
conda activate hds_stoch?

## Scenarios
-	Changing number of funded beds, 
-	Ring-fencing beds for Emergency and Elective demand, 
-	Eradicating bed-blocking, 
-	Changing the scheduling of surgery

## Outcomes
After running the simulation, the following outcomes are obtained:
- mean beds in queue:	
- mean beds occupied:
- occupancy rate:	
- mean waiting hours:
- total admissions:
- number of cancellations:
- number of cancellations avoided:
- cost per avoided cancellation:
- number of additional emergency patients admitted:

I just copied the model I got in the 2nd round below, when we finish, we can update the model here.

In [ ]:
import simpy
import numpy as np
import itertools
from distributions import *
import pandas as pd


# Helper function to print out messages    
def trace(msg):
    '''
    Turning printing of events on and off.
    
    Params:
    -------
    msg: str
        string to print to screen.
    '''
    if TRACE:
        print(msg)

In [ ]:
class Scenario:
    '''
    Parameter class for CCU simulation model inputs.
    '''
    def __init__(self):
        '''
        The init method sets up our defaults. 
        '''
        self.beds = simpy.Resource(env, capacity=N_BEDS)
        
        # Inter-arrival time (IAT) distributions for five types of patients
        self.ae_arrival_dist = Exponential(MEAN_IAT_ae, random_seed=SEEDS[0])
        self.ward_arrival_dist = Exponential(MEAN_IAT_ward, random_seed=SEEDS[1])
        self.emer_arrival_dist = Exponential(MEAN_IAT_emer, random_seed=SEEDS[2])
        self.oth_arrival_dist = Exponential(MEAN_IAT_oth, random_seed=SEEDS[3])
        self.xray_arrival_dist = Exponential(MEAN_IAT_xray, random_seed=SEEDS[4])

        # Length of stay (LOS) distributions for six types of patients
        self.ae_los_dist = Lognormal(MEAN_LOS_ae, STD_LOS_ae, random_seed=SEEDS[5])
        self.ward_los_dist = Lognormal(MEAN_LOS_ward, STD_LOS_ward, random_seed=SEEDS[6])
        self.emer_los_dist = Lognormal(MEAN_LOS_emer, STD_LOS_emer, random_seed=SEEDS[7])
        self.oth_los_dist = Lognormal(MEAN_LOS_oth, STD_LOS_oth, random_seed=SEEDS[8])
        self.xray_los_dist = Lognormal(MEAN_LOS_xray, STD_LOS_xray, random_seed=SEEDS[9])
        self.es_los_dist = Lognormal(MEAN_LOS_es, STD_LOS_es, random_seed=SEEDS[10])

In [ ]:
class Patient:
    '''
    Patient in the CCU
    '''
    def __init__(self, identifier, env, source, args):
        '''
        Constructor method
        
        Params:
        -----
        identifier: int
            a numeric identifier for the patient.
            
        env: simpy.Environment
            the simulation environment
            
        args: Scenario
            The input data for the scenario
        '''
        # patient and environment
        self.identifier = identifier
        self.env = env        
        self.source = source        
        self.beds = args.beds
        

        # Length of stay (LOS) distributions for five types of patients
        self.ae_los_dist = args.ae_los_dist
        self.ward_los_dist = args.ward_los_dist
        self.emer_los_dist = args.emer_los_dist
        self.oth_los_dist = args.oth_los_dist
        self.xray_los_dist = args.xray_los_dist
        
        # individual parameter
        self.wait_time = 0.0


    def service(self):
        '''
        simulates the process for unplanned admissions in CCU 
        
        1. request and wait for a bed
        2. stay in CCU for a period of LOS
        3. exit system.
        
        '''
        # record the time that patient entered the system
        arrival_time = self.env.now

        # request a bed 
        with self.beds.request() as req:
            yield req
            
            # waiting time
            self.wait_time = self.env.now - arrival_time
            
            # sample LOS
            self.los = self.sample_los()
            trace(f'Patient {self.identifier} from {self.source} waited for {self.wait_time:.2f} hours. '\
                  + f'LOS: {self.los:.2f}')
            
            yield self.env.timeout(self.los)            
            
            trace(f'Patient {self.identifier} from {self.source} left at {self.env.now:.2f}')
            
            
    def sample_los(self):
        '''
        Sample the LOS distribution 
        according to different type of sources.
        '''
        if self.source == 'A&E':
            self.los = self.ae_los_dist.sample()
        elif self.source == 'Ward':
            self.los = self.ward_los_dist.sample()
        elif self.source == 'Emergency':
            self.los = self.emer_los_dist.sample()
        elif self.source == 'Other Hospital':
            self.los = self.oth_los_dist.sample()
        elif self.source == 'X-ray':
            self.los = self.xray_los_dist.sample()
                        
        return self.los

In [ ]:
class ElectivePatient(Patient):
    '''
    Elective surgery patient in the CCU
    '''
    # Track patients who cancel surgery
    cancelled_surgeries = []
          
    def __init__(self, identifier, env, source, args):
        '''
        Constructor method
        
        Params:
        -----
        identifier: int
            a numeric identifier for the patient.
            
        env: simpy.Environment
            the simulation environment
            
        args: Scenario
            The input data for the scenario
        '''
        super().__init__(identifier, env, source, args)
        self.es_los_dist = args.es_los_dist
        
        self.los = 0.0
    
    @classmethod
    def reset_cancellations(cls):
        cls.cancelled_surgeries = []
    
    def service(self):
        '''
        simulates the process for planned admissions in CCU 
        
        1. request a bed or cancel the surgery
        2. stay in CCU for a period of LOS
        3. exit system.
        
        '''
        # record the time that patient entered the system
        arrive_time = self.env.now

        # check if there is available bed
        if self.beds.count < self.beds.capacity:
            # request a bed
            with self.beds.request() as req:
                yield req
                # sample LOS
                self.los = self.es_los_dist.sample()
                trace(f'Patient {self.identifier} from {self.source}'\
                      + f' admitted with LOS: {self.los:.2f}')
                
                yield self.env.timeout(self.los)
                
                trace(f'Patient {self.identifier} from {self.source}'\
                      + f' left at {self.env.now:.2f}')
        else:
            # Add in the calcelled list
            ElectivePatient.cancelled_surgeries.append(self.identifier)
            
            trace(f'Patient {self.identifier} from {self.source}'\
                  + f' cancelled at {arrive_time:.2f}')
            

In [ ]:
class CCU:  
    '''
    Model of a CCU
    '''
    def __init__(self, env, args):
        '''
        Contructor
        
        Params:
        -------
        env: simpy.Environment
        
        args: Scenario
            container class for simulation model inputs.
        '''
        self.env = env
        self.args = args
        
        self.ae_arrival_dist = args.ae_arrival_dist
        self.ward_arrival_dist = args.ward_arrival_dist
        self.emer_arrival_dist = args.emer_arrival_dist
        self.oth_arrival_dist = args.oth_arrival_dist
        self.xray_arrival_dist = args.xray_arrival_dist
        
        self.patients = []
        
            
    def ae_arrivals_generator(self):
        '''
        IAT generator for ae patients
        '''
        while True:
            inter_arrival_time = self.ae_arrival_dist.sample()
            yield self.env.timeout(inter_arrival_time)
            
            patient_count = next(identifier_generator)
            trace(f'Patient {patient_count} from A&E'\
                  + f' arrived at {self.env.now:.2f}')

            # create a new patient and pass in env and args
            new_patient = Patient(patient_count, self.env, 'A&E', self.args)
            
            # keep a record of the patient for results calculation
            self.patients.append(new_patient)
            
            # init the service process for this patient
            self.env.process(new_patient.service())
            
            
    def ward_arrivals_generator(self):
        '''
        IAT generator for ward patients
        '''
        while True:
            inter_arrival_time = self.ward_arrival_dist.sample()
            yield self.env.timeout(inter_arrival_time)

            patient_count = next(identifier_generator)
            trace(f'Patient {patient_count} from Ward'\
                  + f' arrived at {self.env.now:.2f}')

            # create a new patient and pass in env and args
            new_patient = Patient(patient_count, self.env, 'Ward', self.args)
            
            # keep a record of the patient for results calculation
            self.patients.append(new_patient)
            
            # init the service process for this patient
            self.env.process(new_patient.service())
            
            
    def emer_arrivals_generator(self):
        '''
        IAT generator for emergency patients
        '''
        while True:
            inter_arrival_time = self.emer_arrival_dist.sample()
            yield self.env.timeout(inter_arrival_time)

            patient_count = next(identifier_generator)
            trace(f'Patient {patient_count} from Emergency'\
                  + f' arrived at {self.env.now:.2f}')

            # create a new patient and pass in env and args
            new_patient = Patient(patient_count, self.env, 'Emergency', self.args)
            
            # keep a record of the patient for results calculation
            self.patients.append(new_patient)
            
            # init the service process for this patient
            self.env.process(new_patient.service())
            
            
    def oth_arrivals_generator(self):
        '''
        IAT generator for other hospital patients
        '''
        while True:
            inter_arrival_time = self.oth_arrival_dist.sample()
            yield self.env.timeout(inter_arrival_time)

            patient_count = next(identifier_generator)
            trace(f'Patient {patient_count} from Other Hospital'\
                  + f' arrived at {self.env.now:.2f}')

            # create a new patient and pass in env and args
            new_patient = Patient(patient_count, self.env, 'Other Hospital', self.args)
            
            # keep a record of the patient for results calculation
            self.patients.append(new_patient)
            
            # init the service process for this patient
            self.env.process(new_patient.service())
            
            
    def xray_arrivals_generator(self):
        '''
        IAT generator for xray patients
        '''
        while True:
            inter_arrival_time = self.xray_arrival_dist.sample()
            yield self.env.timeout(inter_arrival_time)

            patient_count = next(identifier_generator)
            trace(f'Patient {patient_count} from X-ray'\
                  + f' arrived at {self.env.now:.2f}')

            # create a new patient and pass in env and args
            new_patient = Patient(patient_count, self.env, 'X-ray', self.args)
            
            # keep a record of the patient for results calculation
            self.patients.append(new_patient)
            
            # init the service process for this patient
            self.env.process(new_patient.service())
            
            
    def es_arrivals_generator(self, num_weeks, daily_sample_size):
        '''
        Arrival times generator for elective surgery patients
        '''
        ElectivePatient.reset_cancellations()
        
        while True:
            for week in range(num_weeks):
                for day_of_week in range(7):  
                    # Calculate the absolute day 
                    current_day = week * 7 + day_of_week  

                    # Check for weekdays
                    if 0 <= day_of_week <= 4:  
                        # Sample the arrival times 
                        daily_arrival_times = sample_daily_arrival_times(daily_sample_size, 
                                                                         random_seed=current_day)

                        # calculate the scheduled arrival times
                        last_arrival_time = current_day * 24  
                        for arrival_time in daily_arrival_times:
                            actual_arrival_time = last_arrival_time + arrival_time
                            
                            yield self.env.timeout(actual_arrival_time - env.now)
                            patient_count = next(identifier_generator)
                            trace(f'Patient {patient_count} from Elective Surgery'\
                                  + f' arrived at {self.env.now:.2f}')
                            
                            # create a new patient and pass in env and args
                            new_patient = ElectivePatient(patient_count, self.env, 'Elective Surgery', self.args)
                            
                            # keep a record of the patient for results calculation
                            self.patients.append(new_patient)

                            # init the service process for this patient
                            self.env.process(new_patient.service())
                            
                    else:
                        # skip the weekends
                        continue

In [ ]:
class Auditor:
    def __init__(self, env, run_length, bed_counts, first_obs=None, interval=None):
        '''
        Auditor Constructor
        
        Params:
        -----
        env: simpy.Environment
            
        first_obs: float, optional (default=None)
            Time of first scheduled observation.  If none then no scheduled
            audit will take place
        
        interval: float, optional (default=None)
            Time period between scheduled observations. If none then no scheduled
            audit will take place
        '''
        self.env = env
        self.first_observation = first_obs
        self.interval = interval
        self.run_length = run_length
        self.bed_counts = bed_counts
        
        self.queues = []
        self.services = []
        
        # dict to hold states
        self.metrics = {}
        
        # scheduled the periodic audits
        if not first_obs is None:
            env.process(self.scheduled_observation())
            env.process(self.process_end_of_run())
            
    def add_resource_to_audit(self, resource, name, audit_type='qs'):
        if 'q' in audit_type:
            self.queues.append((name, resource))
            self.metrics[f'queue_length_{name}'] = []
        
        if 's' in audit_type:
            self.services.append((name, resource))
            self.metrics[f'occupied_{name}'] = []   
                    
    def record_queue_length(self):
        for name, res in self.queues:
            self.metrics[f'queue_length_{name}'].append(len(res.queue)) 
               
    def record_occupied_bed(self):
        for name, res in self.services:
            self.metrics[f'occupied_{name}'].append(res.count) 

            
    def scheduled_observation(self):
        '''
        simpy process to control the frequency of 
        auditor observations of the model.  
        
        The first observation takes place at self.first_obs
        and subsequent observations are spaced self.interval
        apart in time.
        '''
        # delay first observation
        yield self.env.timeout(self.first_observation)
        self.record_queue_length()
        self.record_occupied_bed()
        
        while True:
            yield self.env.timeout(self.interval)
            self.record_queue_length()
            self.record_occupied_bed()
               
        
    def process_end_of_run(self):
        '''
        Create an end of run summary
        
        Returns:
        ---------
            pd.DataFrame
        '''
        
        yield self.env.timeout(self.run_length - 1)
        
        run_results = {}

        for name, res in self.queues:
            queue_length = np.array(self.metrics[f'queue_length_{name}'])
            run_results[f'mean_queue_{name}'] = queue_length.mean()
            
        for name, res in self.services:
            serviced_beds = np.array(self.metrics[f'occupied_{name}'])
            run_results[f'mean_occupied_{name}'] = serviced_beds.mean()
            run_results[f'occupancy_rate_{name}'] = (serviced_beds.mean() / self.bed_counts) 

        self.summary_frame = pd.Series(run_results).to_frame()
        self.summary_frame.columns = ['estimate']        

In [ ]:
# collect performance metrics
def run_results(model, auditor):
    df_results = auditor.summary_frame
    
    # admissions from various sources
    ae_admissions = sum(patient.source == 'A&E' for patient in model.patients)
    ward_admissions = sum(patient.source == 'Ward' for patient in model.patients)
    emer_admissions = sum(patient.source == 'Emergency' for patient in model.patients)
    oth_admissions = sum(patient.source == 'Other Hospital' for patient in model.patients)
    xray_admissions = sum(patient.source == 'X-ray' for patient in model.patients)
    
    # Calculate the number of cancelled elective surgery patients
    cancelled_es = len(ElectivePatient.cancelled_surgeries)
    es_admissions = sum(patient.source == 'Elective Surgery' for patient in model.patients) - cancelled_es
    
    # total admissions
    total_admissions = len(model.patients) - cancelled_es
        
    # waiting time = sum(waiting times) / no. patients
    mean_wait_time = np.array([patient.wait_time 
                                for patient in model.patients]).mean()
    
    # bed days utilisation = sum(los) / (run length X no. beds)
    bed_day_util = np.array([patient.los 
                     for patient in model.patients]).sum() / \
                    (RUN_LENGTH * N_BEDS)

    # append to results df
    new_row = pd.DataFrame({'estimate':{'Total_admissions': total_admissions,
                                        'A&E_admissions': ae_admissions,
                                        'Ward_admissions': ward_admissions,
                                        'Emergency_admissions': emer_admissions,
                                        'Other_hospital_admissions': oth_admissions,
                                        'Xray_admissions': xray_admissions,
                                        'Elective_Surgery_admissions': es_admissions,
                                        'Cancelled_Surgeries': cancelled_es, 
                                        'mean_wait_hours': mean_wait_time, 
                                        'bed_days_util': bed_day_util}})

    df_results = pd.concat([df_results, new_row])
    return df_results

In [ ]:
# Run the simulation model
##########################################################
# scheduled audit intervals in hours.
FIRST_OBS = 24
OBS_INTERVAL = 48
N_BEDS = 24

# RUN lENGTH
RUN_LENGTH = 12 * 30 * 24  # 12 months

# Patient inter-arrival time (IAT) distributions
MEAN_IAT_ae = 22.72
MEAN_IAT_ward = 26.0
MEAN_IAT_emer = 37.0
MEAN_IAT_oth = 47.2
MEAN_IAT_xray = 575.0

# Patient length of stay (LOS) distributions
MEAN_LOS_ae = 128.79
STD_LOS_ae = 267.51
MEAN_LOS_ward = 177.89 
STD_LOS_ward = 276.54
MEAN_LOS_emer = 140.15 
STD_LOS_emer = 218.02
MEAN_LOS_oth = 212.86
STD_LOS_oth = 457.67
MEAN_LOS_xray = 87.53
STD_LOS_xray = 108.15
MEAN_LOS_es = 57.34
STD_LOS_es = 99.78

# Elective surgery parameters
DAILY_SAMPLE_SIZE = int(1182 / 12 / 30) # Change the sample size
NUM_WEEKS = math.ceil(RUN_LENGTH / 24 / 7)

# SEEDS to reproduce results of a single run
REPRODUCIBLE_RUN = True    
if REPRODUCIBLE_RUN:
    SEEDS = [42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]
else:
    SEEDS = [None, None, None, None, None, None, None, None, None, None]


# Turn off tracing
TRACE = False
##########################################################

# generate patient identifier
identifier_generator = itertools.count()

# create simpy environment
env = simpy.Environment()

# base case scenario with default parameters
default_args = Scenario()

# instantiate an auditor
auditor = Auditor(env, RUN_LENGTH, N_BEDS, FIRST_OBS, OBS_INTERVAL)
auditor.add_resource_to_audit(default_args.beds, 'beds')

# create the model
model = CCU(env, default_args)

# setup the process
env.process(model.ae_arrivals_generator())
env.process(model.ward_arrivals_generator())
env.process(model.emer_arrivals_generator())
env.process(model.oth_arrivals_generator())
env.process(model.xray_arrivals_generator())
env.process(model.es_arrivals_generator(NUM_WEEKS, DAILY_SAMPLE_SIZE))

env.run(until=RUN_LENGTH)
print(f'End of run. simulation clock time = {env.now}')

print('\nSingle run results\n------------------')
run_results(model, auditor).round(2)

End of run. simulation clock time = 8640

Single run results
------------------


,estimate
mean_queue_beds,1.98
mean_occupied_beds,22.56
occupancy_rate_beds,0.94
A&E_admissions,387.00
Cancelled_Surgeries,456.00
Elective_Surgery_admissions,318.00
Emergency_admissions,254.00
Other_hospital_admissions,186.00
Total_admissions,1485.00
Ward_admissions,321.00
